# Introduction

This notebook shows how to use the confluent-weaviate connector with [Weaviate Cloud Services](https://weaviate.io/pricing)

# Imports

In [1]:
import json
import os
import time

import weaviate
from pyspark.sql import SparkSession

# Setup

Setup weaviate client to connect to Weaviate Cloud Services:

In [2]:
client = weaviate.Client(
    url=os.getenv("WCS_URL"),
    auth_client_secret=weaviate.AuthApiKey(os.getenv("WCS_API_KEY")),
)

client.schema.delete_all()
weaviate_url = client._connection.url
weaviate_host = weaviate_url.split("://")[1]

token = client._connection._headers["authorization"]
weaviate_api_key = token.split("Bearer ")[1]

Setup the spark session:

In [3]:
jar_packages = [
    "org.apache.spark:spark-avro_2.12:3.4.1",
    "org.apache.spark:spark-sql-kafka-0-10_2.12:3.4.1",
]

CONFLUENT_WEAVIATE_JAR = "../target/scala-2.12/confluent-connector_2.12-3.4.0_0.0.1.jar"

spark = (
    SparkSession.builder.appName("demo-confluent-weaviate-integration")
    .config("spark.jars.packages", ",".join(jar_packages))
    .config("spark.jars", CONFLUENT_WEAVIATE_JAR)
    .config("spark.streaming.stopGracefullyOnShutdown", "true")
    .getOrCreate()
)

:: loading settings :: url = jar:file:/home/vscode/.local/lib/python3.9/site-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/vscode/.ivy2/cache
The jars for the packages stored in: /home/vscode/.ivy2/jars
org.apache.spark#spark-avro_2.12 added as a dependency
org.apache.spark#spark-sql-kafka-0-10_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-f617fe18-9527-4cda-b30d-3a2cf33b4b35;1.0
	confs: [default]
	found org.apache.spark#spark-avro_2.12;3.4.1 in central
	found org.tukaani#xz;1.9 in central
	found org.apache.spark#spark-sql-kafka-0-10_2.12;3.4.1 in central
	found org.apache.spark#spark-token-provider-kafka-0-10_2.12;3.4.1 in central
	found org.apache.kafka#kafka-clients;3.3.2 in central
	found org.lz4#lz4-java;1.8.0 in central
	found org.xerial.snappy#snappy-java;1.1.10.1 in central
	found org.slf4j#slf4j-api;2.0.6 in central
	found org.apache.hadoop#hadoop-client-runtime;3.3.4 in central
	found org.apache.hadoop#hadoop-client-api;3.3.4 in central
	found commons-logging#commons-logging;1.1.3 in central
	found com.google.code.find

Grab the creds:

In [4]:
confluentClusterName = os.environ.get("CONFLUENT_CLUSTER_NAME")
confluentBootstrapServers = os.environ.get("CONFLUENT_BOOTSTRAP_SERVERS")
confluentTopicName = os.environ.get("CONFLUENT_TOPIC_NAME")
schemaRegistryUrl = os.environ.get("CONFLUENT_SCHEMA_REGISTRY_URL")
confluentApiKey = os.environ.get("CONFLUENT_API_KEY")
confluentSecret = os.environ.get("CONFLUENT_SECRET")
confluentRegistryApiKey = os.environ.get("CONFLUENT_REGISTRY_API_KEY")
confluentRegistrySecret = os.environ.get("CONFLUENT_REGISTRY_SECRET")

# Demo

Create the schema in Weaviate:

In [5]:
with open("../src/it/resources/schema.json", "r") as f:
    weaviate_schema = json.load(f)

client.schema.create_class(weaviate_schema)

Create a Spark Structured Streaming `DataFrame` to read streaming data from a Confluent Kafka topic:

In [6]:
clickstreamDF = (
    spark.readStream.format("kafka")
    .option("kafka.bootstrap.servers", confluentBootstrapServers)
    .option("subscribe", confluentTopicName)
    .option("startingOffsets", "latest")
    .option("kafka.security.protocol", "SASL_SSL")
    .option(
        "kafka.sasl.jaas.config",
        "org.apache.kafka.common.security.plain.PlainLoginModule required username='{}' password='{}';".format(
            confluentApiKey, confluentSecret
        ),
    )
    .option("kafka.ssl.endpoint.identification.algorithm", "https")
    .option("kafka.sasl.mechanism", "PLAIN")
    .option("failOnDataLoss", "false")
    .option("name", "clickStreamReadFromConfluent")
    .load()
)

Define a function to run on each microbatch:

In [7]:
total_rows_processed = 0


def f(df, batch_id):
    global total_rows_processed
    row_count = df.count()
    total_rows_processed += row_count

    print(f"Number of rows in the batch with batch id {batch_id}: {row_count}")
    df.write.format("io.weaviate.confluent.Weaviate").option("batchsize", 200).option(
        "scheme", "http"
    ).option("host", weaviate_host).option("apiKey", weaviate_api_key).option(
        "className", weaviate_schema["class"]
    ).option(
        "schemaRegistryUrl", schemaRegistryUrl
    ).option(
        "schemaRegistryApiKey", confluentRegistryApiKey
    ).option(
        "schemaRegistryApiSecret", confluentRegistrySecret
    ).mode(
        "append"
    ).save()

Start writing the stream:

In [8]:
query = (
    clickstreamDF.writeStream.foreachBatch(f)
    .queryName("write_stream_to_weaviate")
    .start()
)

23/09/15 13:57:45 WARN ResolveWriteToStream: Temporary checkpoint location created which is deleted normally when the query didn't fail: /tmp/temporary-d5383248-e1e3-45b8-a4d2-d3394c622e60. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.
23/09/15 13:57:45 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.


23/09/15 13:57:46 WARN AdminClientConfig: These configurations '[key.deserializer, value.deserializer, enable.auto.commit, max.poll.records, auto.offset.reset]' were supplied but are not used yet.


Number of rows in the batch with batch id 0: 0


Number of rows in the batch with batch id 1: 4


Number of rows in the batch with batch id 2: 9


Number of rows in the batch with batch id 3: 12


Number of rows in the batch with batch id 4: 11


Stop writing after 15 seconds:

In [9]:
# this does not gracefully shutdown the stream!
# easiest way to gracefully shutdown is to pause the source connector
time.sleep(15)
query.stop()

Compare the number of rows processed and the number of objects in Weaviate:

In [10]:
results = client.query.aggregate(weaviate_schema["class"]).with_meta_count().do()
total_objects_in_weaviate = results["data"]["Aggregate"][weaviate_schema["class"]][0][
    "meta"
]["count"]

assert (
    total_rows_processed == total_objects_in_weaviate
), f"Total rows processed {total_rows_processed} does not match total objects in weaviate {total_objects_in_weaviate}"

Look at some of the objects in Weaviate:

In [11]:
client.data_object.get(class_name=weaviate_schema["class"], limit=3)

{'deprecations': [],
 'objects': [{'class': 'Clickstream',
   'creationTimeUnix': 1694786286944,
   'id': '00fa91d1-453d-4d6f-8b05-5ea56fd81a75',
   'lastUpdateTimeUnix': 1694786286944,
   'properties': {'_kafka_key': '202882',
    '_kafka_offset': 34052,
    '_kafka_partition': 2,
    '_kafka_schemaId': 100002,
    '_kafka_timestamp': '2023-09-15T13:58:01.245Z',
    '_kafka_timestampType': 0,
    '_kafka_topic': 'clickstreams-users',
    'city': 'Palo Alto',
    'first_name': 'Elwyn',
    'last_name': 'Vanyard',
    'level': 'Platinum',
    'registered_at': 1502262440871,
    'user_id': 202882,
    'username': 'AbdelKable_86'},
   'vectorWeights': None},
  {'class': 'Clickstream',
   'creationTimeUnix': 1694786281826,
   'id': '08b21fc6-fad8-405a-851a-4200dd15059d',
   'lastUpdateTimeUnix': 1694786281826,
   'properties': {'_kafka_key': '202871',
    '_kafka_offset': 34048,
    '_kafka_partition': 2,
    '_kafka_schemaId': 100002,
    '_kafka_timestamp': '2023-09-15T13:57:57.294Z',
  